In [1]:
import numpy as np
import pandas as pd

In [25]:
"""
Inputs de muestra
"""

fecha_accidente = "2020-03"

sueldos = [
            58000,
            58000,
            68000,
            70000,
            65000,
            65000,
            65000,
            65000,
            65000,
            75000,
            70000,
            70000,           
]
sueldos = list(filter(None, sueldos))
sueldos = list(map(float,sueldos))

edad = 28

incapacidad_actual = 33

tipo_accidente = 'in_itinere'#'in_itinere','en_ocasion'

In [12]:
"""
Preparación del dataframe general de ripte
"""

ripte = pd.read_csv('https://github.com/Demian762/Calculadora_RIPTE/blob/main/ripte.csv')
ripte['fechas'] = ripte['fechas'].astype('datetime64')
ripte['fechas'] = ripte['fechas'].dt.to_period('M')

In [17]:
"""
Creacion y merge del dataframe particular del cliente
"""

cliente = pd.DataFrame(sueldos)
cliente = cliente.rename(columns={0:'sueldos'})

cliente['fechas'] = pd.date_range(end=fecha_accidente,freq='M',periods=len(sueldos))
cliente['fechas'] = cliente['fechas'].dt.to_period('M')

cliente = pd.merge(cliente, ripte, on='fechas', how='left')
cliente = pd.merge(cliente, tasas, on="fechas", how="left")

indice = ripte[ripte['fechas']==fecha_accidente].final
indice = float(indice)

cliente['coeficiente'] = 0.0

cliente = cliente.assign(coeficiente = lambda x: (indice/x['final']))
cliente = cliente.assign(actualizados = lambda x: (x['sueldos']*x['coeficiente']))
cliente = cliente.round(2)

In [18]:
"""
Resultado final
"""

resultado = cliente['actualizados'].mean().round(2)

In [15]:
"""
Cargando las tasas de una pagina random porque al BNA le chupa un huevo
"""
tasas = pd.read_csv('https://github.com/Demian762/Calculadora_RIPTE/blob/main/tasas.csv')
tasas['fechas'] = tasas['fechas'].astype('datetime64')
tasas['fechas'] = tasas['fechas'].dt.to_period('M')

In [19]:
"""
Calculo final con intereses
"""

fecha_calculo_final = cliente['fechas'].max()

interes = tasas[tasas['fechas'] > fecha_calculo_final]['tasas'].sum()/100 + 1

resultado = (resultado * interes).round(2)

resultado_ci = 53 * resultado *  (65/edad) * incapacidad_actual/(100)
resultado_ci = round(resultado_ci,2)

if tipo_accidente == 1:
    resultado_ci_final = round(resultado_ci*1.2,2)
else:
    resultado_ci_final = resultado_ci


In [23]:
cliente[['fechas','sueldos','indice','actualizados']]

,fechas,sueldos,indice,actualizados
0,2019-11,58000.0,5554.15,77294.45
1,2019-12,65000.0,5666.48,84905.91
2,2020-01,60000.0,6066.07,73211.91
3,2020-02,60000.0,6445.13,68906.07
4,2020-03,63000.0,6500.72,71732.67
5,2020-04,63000.0,6510.18,71628.44
6,2020-05,63000.0,6521.87,71500.05
7,2020-06,72000.0,6670.93,79888.46
8,2020-07,65000.0,6908.52,69641.20
9,2020-08,65000.0,6945.86,69266.82
